In [95]:
import numpy as np
import base64

In [140]:
SCOPE = 10**4

class Encryption:
    def __init__(self):
        self.graph = np.full((SCOPE, SCOPE), '*', dtype='S1')
        self.shape = None
    
    def generate_data_stream(self, path):
        with open(path, "rb") as img2str:
            data = base64.b64encode(img2str.read())
        
        return data

    def regenerate_data_stream(self, data, filename):
        img = base64.b64decode(data)
        decrypted = open(f'decrypted/{filename}', "wb")
        decrypted.write(img)
        decrypted.close()
        return True
    
    def encrypt(self, path=None):
        np.random.seed(42)
        if path == None:
            path = input('Enter path to image file: ')
        try:
            data = self.generate_data_stream(path)
        except:
            return False
        
        # a
        initial_term = np.random.randint(1, SCOPE)
        
        # a + (n-1)d
        ap_base = np.random.randint(2, SCOPE)
        ap_multiple = 1
        
        # d^r
        gp_base = np.random.randint(2, 7)
        gp_power = 0
        
        while len(data) > 0:
            ap_term = initial_term + (ap_multiple-1)*ap_base
            ap_multiple += 1
            gp_term = (gp_base**gp_power)
            gp_power += 1
            u = ap_term // SCOPE
            v = ap_term % SCOPE
            if self.graph[u][v] != b'*':
                gp_power -= 1
                continue
            else:
                feed_length = f'{gp_term}'
                it = iter(feed_length) 
                for i in range(v, SCOPE-v):
                    for j, c in zip(range(u, SCOPE-u), it):
                        self.graph[i, j] = c 
                
                feed = data[:gp_term] if len(data) > gp_term else data
                
                it = iter(feed) 
                for i in range(u, SCOPE-u):
                    for j, c in zip(range(v, SCOPE-v), it):
                        self.graph[i, j] = c
                
                data = data[gp_term:] if len(data) > gp_term else b''
        
        encrypted = open('encrypted/encrypted.bin', 'wb')
        encrypted.write(self.graph)
        encrypted.close()
        return True
    
    def decrypt(self, path=None):
        if path == None:
            path = input('Enter path to encrypted file: ')
        
        try:
            encrypted_file = open(path, "rb")
            content = encrypted_file.read()
        except:
            return False
        
        content_array = np.frombuffer(content, dtype = 'S1')
        content_array = content_array.reshape((SCOPE, SCOPE))
        
        return content_array
        

In [139]:
e = Encryption()
e.encrypt('trial.jpg')

True

In [ ]:
e.decrypt('encrypted/encrypted.bin')